In [9]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.cm as cm
import cmocean.cm as cmo
import intake

import numcodecs
try:
    import gribscan
except:
    %pip install gribscan
    import gribscan
import cfgrib
import gc

from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
from dask.utils import format_bytes
from dask.diagnostics import ProgressBar

from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
#from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
import numpy as np # Pythons standard array library
import xarray as xr # Libary to work with labeled n-dimensional data
import dask # Distributed data libary
import dask.distributed
import multiprocessing
from subprocess import run, PIPE
import sys
import os
import warnings
warnings.filterwarnings(action='ignore')
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
import pandas as pd
import eccodes
import healpy


In [10]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")




In [11]:

def get_nest(dx):
    return dx.crs.healpix_order == "nest"


def get_nside(dx):
    return dx.crs.healpix_nside


def attach_coords(ds):
    lons, lats = healpy.pix2ang(
        get_nside(ds), np.arange(ds.dims["cell"]), nest=get_nest(ds), lonlat=True
    )
    return ds.assign_coords(
        lat=(("cell",), lats, {"units": "degree_north"}),
        lon=(("cell",), lons, {"units": "degree_east"}),
    )


In [12]:

def calculate_temporal(data,year,month,day,function,latitude,longitude):
    indexer = ((data.lat>=latitude[0]) & (data.lat<=latitude[1]) & (data.lon>=longitude[0]) & (data.lon<=longitude[1]))
    data = data.where(indexer,drop=True)
    if function == 'precip':
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data.where(data.time.dt.day.isin(day), drop=True)

        data = data * 900 #ICON precip is mm/s in 15 minute blocks. i.e. 60 * 15
        data.attrs['units'] = 'mm hr-1'
        try:
            data = data.resample(time='1H').sum()
        except ValueError:
            pass        
    if function == 'tmean':
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data - 273.15
        data.attrs['units'] = 'C'     

    if function == 'tmax':
        data = data.where(data.time.dt.year.isin(year), drop=True)
        data = data.where(data.time.dt.month.isin(month), drop=True)
        data = data.where(data.time.dt.day.isin(day), drop=True)
        data = data.resample(time='1D').max()

        data = data - 273.15
        data.attrs['units'] = 'C'     
    if function == 'other':
        print(data)
        #data = data.where(data.time.dt.year.isin(year), drop=True)
        #data = data.where(data.time.dt.month.isin(month), drop=True)
        #data = data.where(data.time.dt.month.isin(day), drop=True)       

    dsel = data.load()
    return dsel
    

    
       

In [13]:
#####function that uses all the functions.
def temporal_mean(dataset,year,month,day,function,latitude,longitude):
    
# If you want to change the longitude range from 0:360 to -180:180 to make clipping easier around the prime  meridian
    dataset['lon_adjusted'] = xr.where(dataset.lon > 180, dataset.lon - 360, dataset.lon)
    dataset['lon'] = dataset['lon_adjusted']
    dataset = dataset.reset_coords(names='lon_adjusted', drop=True)
    ###calculate
    calc_var = calculate_temporal(dataset,year,month,day,function,latitude,longitude)
    return calc_var
    #del var_final
    del dataset
    del cal_var
    gc.collect()

In [14]:

res = 'ICON_ngc4008'
var = ['pr'] #for cape, change to 2D_1h_0.25deg precip = pr 'crs' land_sea_mask = 'ocean_fraction_surface'

function = 'precip'
#months = [1,2,3,4,5,6,7,8,9,10,11,12]
months = [2,3]
#years = np.arange(2020, 2049)
years = [2021]
#days = np.arange(1,32,1)
days = np.arange(28,29,1)

latitude = [35,45] #Choose your own lat/lon range 
longitude = [-11,5]

data = cat.ICON.ngc4008(chunks="auto")(time="PT15M",zoom=9).to_dask().pipe(attach_coords)  #PT15M,PT3H,P1D
dataset = data[var]


In [15]:
variable = temporal_mean(dataset,years,months,days,function,latitude,longitude)


In [16]:
print(variable)

<xarray.Dataset>
Dimensions:  (time: 696, cell: 9371)
Coordinates:
  * time     (time) datetime64[ns] 2021-02-28 ... 2021-03-28T23:00:00
    lat      (cell) float64 36.52 36.61 36.7 36.7 ... 41.51 41.61 41.61 41.71
    lon      (cell) float64 4.922 4.834 4.922 4.746 ... 0.0 0.08789 -0.08789 0.0
Dimensions without coordinates: cell
Data variables:
    pr       (time, cell) float32 2.982e-13 2.672e-13 ... 0.2257 0.02432
Attributes:
    units:    mm hr-1
